In [15]:
from pathlib import Path
import subprocess
import os

fmap_dir=Path("/scratch/jdrussell3/104/s11_fmap/")
os.chdir(fmap_dir)
fmaptype = "DTI"

def getrawfiles():
    gen = (rawfmapfile for rawfmapfile in fmap_dir.glob('*.nii') if rawfmapfile.name.__contains__(fmaptype))
    for rawfmapfile in gen:
        print(rawfmapfile)
        if str(rawfmapfile).__contains__('_e1a.'):
            rawfmapfile_2 = str(rawfmapfile)
        elif str(rawfmapfile).__contains__('_e1.'):
            rawfmapfile_1 = str(rawfmapfile)
        else:
            next    
    try:
        rawfmapfile_2
        rawfmapfile_1
    except AttributeError:
        quit

    # if rawfmapfile_2 is None or rawfmapfile_1 is None:
    #     print("No fieldmap files found...")
    #     return None


    real1 = str(rawfmapfile_1 + '[2]')
    real2 = str(rawfmapfile_2 + '[2]')
    imag1 = str(rawfmapfile_1 + '[3]')
    imag2 = str(rawfmapfile_2 + '[3]')     
    subprocess.call(["3dcalc", "-float", "-a", real1, "-b", imag1, "-c", real2, "-d", imag2, "-expr", 
    "(atan2((b*c-d*a),(a*c+b*d)))", "-prefix", "tmp.wrappedphasediff.nii.gz"])

    subprocess.call(["3dcalc", "-a", str(rawfmapfile_1 + '[0]'), "-expr", "a", "-prefix", "magnitude1.nii.gz"])

    subprocess.call(["bet2", "magnitude1.nii.gz", "magnitude1_brain.nii.gz"])

    subprocess.call(["fslmaths", "magnitude1_brain.nii.gz", "-ero", "magnitude1_brain_ero.nii.gz"])

    subprocess.call(["3dresample", "-master", "tmp.wrappedphasediff.nii.gz", "-prefix", "magnitude1_brain_ero_reg.nii.gz", "-input", "magnitude1_brain_ero.nii.gz"])
    print("test")
    subprocess.call(["prelude", "-v", "-p", "tmp.wrappedphasediff.nii.gz", "-a", "magnitude1_brain_ero.nii.gz", "-o", "tmp.phasediff.rads.nii.gz", 
    "-m", "magnitude1_brain_ero_reg.nii.gz"])
    print("test2")
    subprocess.call(["3dresample", "-input", "tmp.phasediff.rads.nii.gz", "-prefix", "DTI_RealFieldmap_rads.nii.gz", "-orient", "LPI"])


    subprocess.call(["3dcalc", "-a", "DTI_RealFieldmap_rads.nii.gz", "-expr", "a*0.1592", "-prefix", "DTI_RealFieldmap_Hz.nii.gz"])
    
getrawfiles()

/scratch/jdrussell3/104/s11_fmap/__Fieldmap_(DTI)_20140507172900_11_e1a.nii
/scratch/jdrussell3/104/s11_fmap/__Fieldmap_(DTI)_20140507172900_11_e1.nii
test
test2
